<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Logging%2C_Visualisation_%26_Statistical_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === secure experiment logger with end-to-end single-key encryption ===
# pip install cryptography torch tensorboard pandas

import os
import base64
import secrets
import hashlib
import json
import datetime
import pathlib
from typing import Optional, Iterable, Dict, Any

import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

# ----------------------------
# Configuration and constants
# ----------------------------
ALG = "AES-256-GCM"
ENV_KEY = "EXPLOG_KEY_B64"   # Base64-encoded 32-byte key
KEYFILE_NAME = "key.b64"     # Fallback key file inside the run directory
ENC_FILENAME = "episodes.jsonl.enc"
MANIFEST = "manifest.json"

# ----------------------------
# Key management
# ----------------------------
def _decode_key(b64key: str) -> bytes:
    raw = base64.b64decode(b64key)
    if len(raw) != 32:
        raise ValueError("Key must be 32 bytes (Base64 of 32 bytes for AES-256).")
    return raw

def _encode_key(raw: bytes) -> str:
    return base64.b64encode(raw).decode("ascii")

def generate_key() -> str:
    """Generate a new 32-byte key and return it as Base64."""
    return _encode_key(secrets.token_bytes(32))

def load_or_create_key(logdir: pathlib.Path) -> bytes:
    """
    Load key from ENV or logdir/key.b64. If absent, generate one and save to logdir/key.b64.
    Return raw key bytes.
    """
    # 1) ENV takes precedence
    b64 = os.getenv(ENV_KEY)
    if b64:
        key = _decode_key(b64)
        return key

    # 2) Key file inside run directory
    keyfile = logdir / KEYFILE_NAME
    if keyfile.exists():
        b64 = keyfile.read_text().strip()
        key = _decode_key(b64)
        return key

    # 3) Create new key and store it
    b64 = generate_key()
    keyfile.write_text(b64)
    print(f"[secure-logger] Generated new key and saved to: {keyfile}")
    print(f"[secure-logger] To reuse elsewhere, set environment variable:\n  {ENV_KEY}={b64}")
    return _decode_key(b64)

def key_fingerprint(raw_key: bytes) -> str:
    """Short fingerprint to identify the key without revealing it."""
    return hashlib.sha256(raw_key).hexdigest()[:16]

# ----------------------------
# Encryption helpers
# ----------------------------
def encrypt_line(raw_key: bytes, data: bytes) -> Dict[str, str]:
    """
    Encrypt a byte payload; returns a JSON-serializable dict with version, nonce, cipher.
    """
    aes = AESGCM(raw_key)
    nonce = secrets.token_bytes(12)  # 96-bit nonce for GCM
    cipher = aes.encrypt(nonce, data, None)
    return {
        "v": 1,
        "n": base64.b64encode(nonce).decode("ascii"),
        "c": base64.b64encode(cipher).decode("ascii"),
    }

def decrypt_line(raw_key: bytes, rec: Dict[str, str]) -> bytes:
    aes = AESGCM(raw_key)
    nonce = base64.b64decode(rec["n"])
    cipher = base64.b64decode(rec["c"])
    return aes.decrypt(nonce, cipher, None)

# ----------------------------
# Logger setup
# ----------------------------
def create_run_dir() -> pathlib.Path:
    ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    logdir = pathlib.Path("runs") / ts
    logdir.mkdir(parents=True, exist_ok=True)
    return logdir

class SecureLogger:
    def __init__(self, logdir: Optional[pathlib.Path] = None):
        self.logdir = logdir or create_run_dir()
        self.key = load_or_create_key(self.logdir)
        self.fingerprint = key_fingerprint(self.key)

        # Write manifest for provenance
        manifest = {
            "algorithm": ALG,
            "created": datetime.datetime.now().isoformat(timespec="seconds"),
            "key_fingerprint": self.fingerprint,
            "enc_file": ENC_FILENAME,
        }
        (self.logdir / MANIFEST).write_text(json.dumps(manifest, indent=2))

        # TensorBoard (local viewing only; not encrypted)
        self.writer = SummaryWriter(log_dir=self.logdir.as_posix())

        # Encrypted output path
        self.enc_path = self.logdir / ENC_FILENAME

        # In-memory list for live analysis (not saved unless you export)
        self._episode_log = []

    def log_episode(self, ep_idx: int, reward: float, length: int, safety_cost: float):
        row = {
            "episode": ep_idx,
            "reward": reward,
            "length": length,
            "safety": safety_cost,
            "ts": datetime.datetime.now().isoformat(timespec="seconds"),
        }

        # TensorBoard scalars (plaintext on local disk)
        self.writer.add_scalar("reward/episode", reward, ep_idx)
        self.writer.add_scalar("safety/episode", safety_cost, ep_idx)
        self.writer.flush()

        # Encrypted append
        rec = encrypt_line(self.key, json.dumps(row, separators=(",", ":")).encode("utf-8"))
        with open(self.enc_path, "a", encoding="utf-8") as f:
            f.write(json.dumps(rec) + "\n")

        # In-memory
        self._episode_log.append(row)

    def get_episode_df(self) -> pd.DataFrame:
        return pd.DataFrame(self._episode_log)

    def save_episode_csv_encrypted(self, filename: str = "episodes.csv.enc"):
        """
        Export the in-memory DataFrame to CSV, then encrypt and write to disk.
        No plaintext CSV is written.
        """
        df = self.get_episode_df()
        csv_bytes = df.to_csv(index=False).encode("utf-8")
        rec = encrypt_line(self.key, csv_bytes)
        out_path = self.logdir / filename
        out_path.write_text(json.dumps(rec))
        print(f"[secure-logger] Encrypted CSV saved to {out_path}")

# ----------------------------
# Decryption utilities
# ----------------------------
def load_key_from_env_or_file(logdir: pathlib.Path) -> bytes:
    """
    Convenience loader mirroring the logger's key logic.
    """
    return load_or_create_key(logdir)

def iter_decrypted_jsonl(logdir: pathlib.Path, key: Optional[bytes] = None) -> Iterable[Dict[str, Any]]:
    """
    Yield decrypted JSON objects from episodes.jsonl.enc.
    """
    key = key or load_key_from_env_or_file(logdir)
    enc_path = logdir / ENC_FILENAME
    with open(enc_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            rec = json.loads(line)
            plain = decrypt_line(key, rec)
            yield json.loads(plain.decode("utf-8"))

def load_decrypted_df(logdir: pathlib.Path, key: Optional[bytes] = None) -> pd.DataFrame:
    """
    Load the encrypted JSONL back into a DataFrame.
    """
    rows = list(iter_decrypted_jsonl(logdir, key))
    return pd.DataFrame(rows)

def decrypt_csv_file(enc_csv_path: pathlib.Path, key: Optional[bytes] = None) -> pd.DataFrame:
    """
    Decrypt a single-record encrypted CSV (from save_episode_csv_encrypted) and return a DataFrame.
    """
    logdir = enc_csv_path.parent
    key = key or load_key_from_env_or_file(logdir)
    rec = json.loads(enc_csv_path.read_text())
    csv_bytes = decrypt_line(key, rec)
    from io import StringIO
    return pd.read_csv(StringIO(csv_bytes.decode("utf-8")))

# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    # Create a secure logger for this run
    logger = SecureLogger()  # or SecureLogger(pathlib.Path("runs/2025..."))

    # Optionally, set a custom key via environment before running:
    # os.environ["EXPLOG_KEY_B64"] = "<Base64 32-byte key>"

    # Log a few dummy episodes
    for ep in range(5):
        logger.log_episode(ep_idx=ep, reward=ep * 1.5, length=100 + ep, safety_cost=ep * 0.1)

    # Export encrypted CSV (optional)
    logger.save_episode_csv_encrypted()

    # Decrypt back into a DataFrame later (same or different machine)
    df = load_decrypted_df(logger.logdir)  # uses ENV or key file
    print(df.tail())

    # If you exported the encrypted CSV:
    csv_df = decrypt_csv_file(logger.logdir / "episodes.csv.enc")
    print(csv_df.head())